In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import keras

from tensorflow.keras.layers import RandomTranslation, RandomZoom, RandomRotation

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D

import matplotlib.pyplot as plt
import seaborn as sns

import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS
from src.visualization.visualize import draw_spider_graph_dark, conf_matrix_dark

## Travail sur un échantilon

In [ ]:
sample = pd.read_parquet(os.path.join(project_root,'data', 'metadata', 'samples', 'df_documents_sample_40k_1.parquet'), engine='fastparquet')

converted_prefix = os.path.join(project_root, 'data', 'converted')
sample['filepath'] = sample['rvl_image_path'].apply(lambda p: os.path.join(converted_prefix, p.replace("raw/", "").replace(".tif", ".jpg")))
sample = sample.drop(columns=['rvl_image_path', 'document_id', 'filename', 'iit_image_path', 'iit_individual_xml_path', 'iit_collective_xml_path'])

## Création des sets

In [ ]:
# 1. Encodage des labels : pas besoin ? 
label_encoder = LabelEncoder()
sample['label_encoded'] = label_encoder.fit_transform(sample['label'])

# 2. On part de sample pour créer les différents sets
df_train = sample[sample['data_set'] == 'train']
df_val = sample[sample['data_set'] == 'val']
df_test = sample[sample['data_set'] == 'test']

# 3. Fonction pour charger et prétraiter une image
def process_image(file_path, label, augment=False):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = preprocess_input(image)
    if augment:
        image = data_augmentation(image)
    return image, label

# 4. Création du dataset
def get_dataset(df_subset, shuffle=False, augment=False):
    file_paths = df_subset['filepath'].values
    labels = df_subset['label_encoded'].values
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: process_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    
    dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    return dataset

train_ds = get_dataset(df_train, shuffle=True)
val_ds = get_dataset(df_val)
test_ds = get_dataset(df_test)

In [ ]:
# Charger la base ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches sauf 10
base_model.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Construction du modèle
inputs = Input(shape=(224, 224, 3))

#On teste quelques augmentations: NON, ça donne des résultats mauvais
#x = tf.keras.layers.RandomRotation(0.02)(inputs)
#x = tf.keras.layers.RandomZoom(0.1)(x)
#x = tf.keras.layers.RandomContrast(0.1)(x)
#x = tf.keras.layers.RandomTranslation(0.05, 0.05)(x)

#on envoie tout ça dans le modèle
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(16, activation='softmax')(x)
model = Model(inputs, outputs)

# Compilation
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(patience=5, min_delta=0.001, verbose=1, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint('best_resnet_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Entraînement
history_1 = model.fit(train_ds,
                      validation_data=val_ds,
                      epochs=10,
                      callbacks=[early_stopping, reduce_lr, checkpoint])

In [ ]:
# On relance après avoir dégelé des couches (il y en a 177 en tout dans ResNet donc là on en dégèle 27 supplémentaires, soit 37

base_model.trainable = True
for layer in base_model.layers[:140]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_2 = model.fit(train_ds,
                      validation_data=val_ds,
                      epochs=10,
                      callbacks=[early_stopping, reduce_lr, checkpoint])

In [ ]:
# encore une étape de dégelage, on rajoute 20 couches à entrainer 

for layer in base_model.layers[:120]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_3 = model.fit(train_ds,
                      validation_data=val_ds,
                      epochs=10,
                      callbacks=[early_stopping, reduce_lr, checkpoint])

In [ ]:
def combine_histories(*histories):
    combined = {}
    for key in histories[0].history.keys():
        combined[key] = sum((h.history[key] for h in histories), [])
    return combined

combined_history = combine_histories(history_1, history_2, history_3)

In [ ]:
# Save the best model
model.save(os.path.join(project_root,'models','ResNet50_best_30_epocs_sample_40_000_unfreeze_step_by_step.keras'))

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(121)
plt.plot(combined_history['loss'])
plt.plot(combined_history['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')

plt.subplot(122)
plt.plot(combined_history['accuracy'])
plt.plot(combined_history['val_accuracy'])
plt.title('Model accuracy by epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')
plt.show()

In [ ]:
model = keras.saving.load_model(os.path.join(project_root,'models','ResNet50_best_30_epocs_sample_40_000_unfreeze_step_by_step.keras'))

In [ ]:
# Étape 1 : Prédire sur le test set
y_pred_probs = model.predict(test_ds)  # Probabilités
y_pred = np.argmax(y_pred_probs, axis=1)  # Classes prédites

# Étape 2 : Récupérer les vrais labels depuis le test set, dans le même ordre
y_true_check = []
for batch in test_ds:
    images, labels = batch
    y_true_check.extend(labels.numpy())

y_true = np.array(y_true_check)

# Étape 3 : Rapport de classification
print("\n Rapport de classification :")
print(classification_report(y_true, y_pred))



In [ ]:
cm = confusion_matrix(y_true, y_pred)
conf_matrix_dark(cm, "illustrations/ResNet_2_cm.png")

In [ ]:
draw_spider_graph_dark(y_true, y_pred, save_path="illustrations/ResNet_2_spider.png")


In [ ]:
# Étape 4 : Matrice de confusion
print("Matrice de confusion :")
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matrice de confusion")
plt.xlabel("Classe prédite")
plt.ylabel("Classe réelle")
plt.show()

# degel progressif automatique: FAILED
Aucune idée pourquoi mais, dès le début, ça n'apprend pas, la loss ne fait qu'augmenter, et redescend par accoups au moment des dégels. 

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Charger la base ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches sauf 10
base_model.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Définir l’architecture complète
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(16, activation='softmax')(x)  # 16 classes
model = Model(inputs, outputs)

# Compilation initiale
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K

class SafeUnfreezeCallback(Callback):
    def __init__(self, base_model, model_path='best_model.keras',
                 unfreeze_step=5, max_unfreeze=30,
                 patience=5, min_delta=0.001):
        super().__init__()
        self.base_model = base_model
        self.model_path = model_path
        self.unfreeze_step = unfreeze_step
        self.max_unfreeze = max_unfreeze
        self.patience = patience
        self.min_delta = min_delta
        self.current_unfrozen = 10
        self.best_val_loss = float('inf')
        self.wait = 0
        self.trigger_reload = False

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        if val_loss is None:
            return

        if val_loss < self.best_val_loss - self.min_delta:
            self.best_val_loss = val_loss
            self.wait = 0
        else:
            self.wait += 1

        if self.wait >= self.patience and self.current_unfrozen < self.max_unfreeze:
            total_layers = len(self.base_model.layers)
            start = max(total_layers - self.current_unfrozen - self.unfreeze_step, 0)
            end = total_layers - self.current_unfrozen
            unfrozen = 0
            for layer in self.base_model.layers[start:end]:
                # Important : ne pas dégeler les BatchNormalization
                if not isinstance(layer, tf.keras.layers.BatchNormalization):
                    layer.trainable = True
                    unfrozen += 1

            self.current_unfrozen += unfrozen
            print(f"\n🔓 Dégel de {unfrozen} couches supplémentaires (total dégelées : {self.current_unfrozen})")

            # Réduction du learning rate
            old_lr = float(K.get_value(self.model.optimizer.learning_rate))
            new_lr = max(old_lr * 0.5, 1e-5)
            try:
                K.set_value(self.model.optimizer.learning_rate, new_lr)
            except AttributeError:
                print("⚠️ Impossible de modifier le learning rate — mauvais type. Recréation de l'optimiseur avec le nouveau LR.")
                self.model.compile(
                    optimizer=tf.keras.optimizers.Adam(learning_rate=new_lr),
                    loss=self.model.loss,
                    metrics=self.model.metrics,
                )
            print(f"📉 Nouveau learning rate : {old_lr:.2e} → {new_lr:.2e}")

            # Stop pour recharger le meilleur modèle
            print("⚠️ Entraînement interrompu → rechargement du meilleur modèle")
            self.model.stop_training = True
            self.trigger_reload = True

    def reset(self):
        self.wait = 0
        self.best_val_loss = float('inf')
        self.trigger_reload = False

In [ ]:
early_stopping = EarlyStopping(patience=5, min_delta=0.001, monitor='val_loss', mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint('best_resnet_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Initialisation de notre callback personnalisé
unfreeze_cb = SafeUnfreezeCallback(base_model=base_model,
                                   unfreeze_step=5,
                                   max_unfreeze=100,
                                   patience=5,
                                   min_delta=0.001)


In [ ]:
max_rounds = 10  # Nombre maximal de phases d'entraînement
combined_history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}


for round_idx in range(max_rounds):
    print(f"\n🔁 Phase d'entraînement {round_idx + 1}")
    
    # 🔄 Réinitialise proprement le callback
    unfreeze_cb.reset()

    # Recharger le meilleur modèle
    model.load_weights('best_resnet_model.keras')
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=float(K.get_value(model.optimizer.learning_rate))),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=15,
                        callbacks=[unfreeze_cb, early_stopping, reduce_lr, checkpoint])

    # Combiner les historiques
    for key in combined_history:
        combined_history[key] += history.history.get(key, [])

    if not unfreeze_cb.trigger_reload:
        print("\n✅ Entraînement terminé — plus de couches à dégeler ou amélioration suffisante.")
        break


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(121)
plt.plot(combined_history['loss'])
plt.plot(combined_history['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')

plt.subplot(122)
plt.plot(combined_history['accuracy'])
plt.plot(combined_history['val_accuracy'])
plt.title('Model accuracy by epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')
plt.show()